# Assignment 3

Name : UjjwaL Akash

Build a CNN model for Bird species

Bird species classification is the process of using machine learning and computer vision techniques to identify and categorize different species of birds based on their visual characteristics. By analyzing images of birds, models can extract features and patterns to accurately classify bird species. This classification is vital for ecological research, wildlife monitoring, and conservation efforts. Advancements in deep learning and the availability of large annotated datasets have improved the accuracy of bird species classification models. Challenges include variations in lighting, pose, and background clutter. Ongoing research focuses on methods like transfer learning and data augmentation to enhance classification performance and contribute to avian biodiversity understanding and conservation.

Dataset Link: https://www.kaggle.com/datasets/akash2907/bird-species-classification

In [ ]:
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model


/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

In [ ]:
dataset_dir = '/kaggle/input/bird-species-classification'


In [ ]:
input_shape = (224, 224, 3)
batch_size = 32


In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2  # 20% of the data will be used for validation
)

train_generator = train_datagen.flow_from_directory(
    os.path.join(dataset_dir, 'train_data', 'train_data'),
    target_size=input_shape[:2],
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    os.path.join(dataset_dir, 'train_data', 'train_data'),
    target_size=input_shape[:2],
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)


Found 124 images belonging to 16 classes.
Found 26 images belonging to 16 classes.


In [ ]:
weights_path = '/kaggle/input/mobilenetmodel/mobilenet_v2_weights_tf_dim_ordering_tf_kernels_1.0_224_no_top.h5'


In [ ]:
base_model = MobileNetV2(
    input_shape=input_shape,
    include_top=False,
    weights=weights_path
)

for layer in base_model.layers:
    layer.trainable = False


In [ ]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu')(x)
predictions = Dense(train_generator.num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)


In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size,
    epochs=20
)


Epoch 1/20
3/3 [==============================] - 19s 7s/step - loss: 0.0675 - accuracy: 1.0000
Epoch 2/20
3/3 [==============================] - 15s 5s/step - loss: 0.0665 - accuracy: 0.9896
Epoch 3/20
3/3 [==============================] - 15s 5s/step - loss: 0.0277 - accuracy: 1.0000
Epoch 4/20
3/3 [==============================] - 15s 5s/step - loss: 0.0460 - accuracy: 0.9891
Epoch 5/20
3/3 [==============================] - 15s 5s/step - loss: 0.0259 - accuracy: 1.0000
Epoch 6/20
3/3 [==============================] - 15s 5s/step - loss: 0.0265 - accuracy: 1.0000
Epoch 7/20
3/3 [==============================] - 14s 5s/step - loss: 0.0203 - accuracy: 1.0000
Epoch 8/20
3/3 [==============================] - 14s 5s/step - loss: 0.0296 - accuracy: 0.9891
Epoch 9/20
3/3 [==============================] - 15s 5s/step - loss: 0.0149 - accuracy: 1.0000
Epoch 10/20
3/3 [==============================] - 16s 5s/step - loss: 0.0082 - accuracy: 1.0000
Epoch 11/20
3/3 [======================

In [ ]:
test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
    os.path.join(dataset_dir, 'test_data', 'test_data'),
    target_size=input_shape[:2],
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False  # Ensure ordering is maintained for evaluation
)


Found 157 images belonging to 16 classes.


In [ ]:
validation_loss, validation_accuracy = model.evaluate(validation_generator)
print('Validation Loss:', validation_loss)
print('Validation Accuracy:', validation_accuracy)


1/1 [==============================] - 4s 4s/step - loss: 1.5101 - accuracy: 0.6538
Validation Loss: 1.510079264640808
Validation Accuracy: 0.6538461446762085


In [ ]:
test_loss, test_accuracy = model.evaluate(test_generator)
print('Test Loss:', test_loss)
print('Test Accuracy:', test_accuracy)


5/5 [==============================] - 30s 5s/step - loss: 2.5471 - accuracy: 0.4968
Test Loss: 2.547107458114624
Test Accuracy: 0.4968152940273285
